In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import psycopg2

con = psycopg2.connect("dbname=test1")

# Data import and process

In [2]:
course_id = "C00198"

In [3]:
userslist = pd.read_sql_query("select user_id from coursesusers where course_id like '%s';" % course_id,con=con)
userslist = [u[0] for u in userslist.values if u[0]]
usersstr = str(userslist).strip('[]')

users = pd.read_sql_query("select user_id, creation_date from users \
                            where demo=0 and instructor=0 and administrator=0\
                            and user_id in (%s);" % usersstr,con=con)

lusers = users.user_id.tolist()
usersstr = str(lusers).strip('[]')

problists = pd.read_sql_query("select list_id from courseslists where course_id like '%s';" % course_id ,con=con)
problists = [l[0] for l in problists.values if l[0]]
problistsstr = str(problists).strip('[]')

probs = pd.read_sql_query("select problem_nm from listitems where list_id \
                                in(%s);" % problistsstr,con=con)

lprobs = [p[0] for p in probs.values if p[0]]
lprobsstr = str(lprobs).strip('[]')

submissions = pd.read_sql_query("select submission_uid, user_id, problem_id, submission_id, \
                        state, time_out, time_in, veredict, score \
                        from submissions where user_id in (%s);" % (usersstr) ,
                       con=con)

submissions.problem_id = submissions.problem_id.apply(lambda x: x[:-3])
submissions.set_index('submission_uid', inplace=True)

submissions = submissions[submissions.problem_id.isin(lprobs)]

goodSubs = submissions[submissions.veredict == 'AC']
badSubs = submissions[submissions.veredict != 'AC']

# Main table creation

We are going to work with a bi dimensional array of booleans

The first dimension is the users ids and the second one is the probs ids.

In [123]:
table = pd.DataFrame(columns=lusers , index=lprobs)
table

,U14913,U14908,U14949,U14903,U14881,U15747,U15826,U15954,U15827,U15969,...,U16564,U16370,U16609,U15828,U16688,U12971,U12942,U00214,U14989,U15243
P46254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
P45616,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
P85696,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
P84584,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
P53361,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
P26141,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
P84786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
P39799,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
P52274,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
P56635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


If a user has correctly resolved a problem we can set that value to **True**

Here is an example with a specific user :

In [115]:
p = goodSubs[(goodSubs.problem_id == 'P86578') & 
             (goodSubs.user_id == 'U14908')]

In [116]:
p.count().user_id

1

Ici l'utilisateur **U14908** à réslut le problème **P86578**

In [117]:
table['U14908']['P86578'] = p.count().user_id

In [118]:
table = pd.DataFrame(index=lprobs)

In [119]:
def checkSub(uid, pid):
    p = goodSubs[(goodSubs.problem_id == pid) & 
             (goodSubs.user_id == uid)]
    n = p.count().user_id
    return n if n else float('NaN')

In [120]:
for u in lusers:
    table[u] = map(lambda x : checkSub(u,x), table.index)

In [121]:
table

,U14913,U14908,U14949,U14903,U14881,U15747,U15826,U15954,U15827,U15969,...,U16564,U16370,U16609,U15828,U16688,U12971,U12942,U00214,U14989,U15243
P46254,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,...,1.0,1.0,NaN,1.0,NaN,1.0,1.0,3.0,NaN,1.0
P45616,2.0,1.0,1.0,NaN,1.0,1.0,NaN,1.0,NaN,1.0,...,NaN,1.0,NaN,1.0,NaN,1.0,1.0,1.0,NaN,1.0
P85696,2.0,2.0,1.0,NaN,1.0,1.0,NaN,1.0,NaN,1.0,...,NaN,1.0,NaN,1.0,NaN,1.0,1.0,6.0,1.0,1.0
P84584,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,1.0,1.0,1.0,1.0,1.0
P53361,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,2.0,...,NaN,1.0,NaN,NaN,NaN,1.0,1.0,2.0,NaN,1.0
P26141,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN
P84786,1.0,1.0,1.0,NaN,1.0,NaN,1.0,NaN,NaN,1.0,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0
P39799,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
P52274,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0
P56635,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN


In [ ]:
pd.SparseArray